In [4]:
#######################################################################
# ENEM 2023 – Predição das 5 notas finais (CN, CH, LC, MT, Redação)
# a partir de variáveis socioeconômicas, usando CatBoost multitarget.
# ---------------------------------------------------------------------
# O código é modular: cada seção pode ser reutilizada ou substituída
# facilmente em outros notebooks / scripts.
#######################################################################

# ==================== 1. IMPORTS  ====================================
import warnings, os, sys
from pathlib import Path
warnings.filterwarnings("ignore")

# --- ciência de dados ---
import pandas as pd
import numpy as np

# --- machine‑learning ---
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor, Pool

print("✔️  Bibliotecas principais importadas")

# ==================== 2. PARAMETROS GLOBAIS ==========================
CSV_PATH      = Path(r"C:\wamp64\www\previsor-enem\dados\MICRODADOS_ENEM_2023.csv")
SAMPLE_FRAC   = 0.03          # 3 % para protótipo (None = usar 100 %)
TEST_SIZE     = 0.20          # 80 / 20 train‑test split
RANDOM_STATE  = 42

# ==================== 3. LISTAS DE COLUNAS ===========================
targets = ["NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC",
           "NU_NOTA_MT", "NU_NOTA_REDACAO"]          # 5 alvos finais
socio_cols = [
    # Dados sociodemográficos / questionário:
    "TP_FAIXA_ETARIA","TP_SEXO","TP_ESTADO_CIVIL","TP_COR_RACA",
    "TP_NACIONALIDADE","TP_ST_CONCLUSAO","TP_ANO_CONCLUIU",
    "TP_ESCOLA","TP_ENSINO","IN_TREINEIRO","CO_MUNICIPIO_ESC",
    "TP_LOCALIZACAO_ESC","TP_SIT_FUNC_ESC",
    # Questionário socioeconômico:
    "Q001","Q002","Q003","Q004","Q005","Q006","Q007","Q008",
    "Q009","Q010","Q011","Q012","Q013","Q014","Q015","Q016",
    "Q017","Q018","Q019","Q020","Q021","Q022","Q023","Q024","Q025",
    # Indicadores de participação (serão retirados depois do filtro):
    "TP_PRESENCA_CN","TP_PRESENCA_CH","TP_PRESENCA_LC","TP_PRESENCA_MT",
    "TP_STATUS_REDACAO"
]

dtypes = {c: "category" for c in socio_cols}

# ==================== 4. LEITURA DO CSV ==============================
df = pd.read_csv(
    CSV_PATH,
    sep=";",
    encoding="latin-1",
    usecols=socio_cols + targets,
    dtype=dtypes,
    engine="pyarrow"        # remova se usar pandas < 2.2
)
print(f"✔️  CSV carregado  |  shape = {df.shape}  "
      f"|  memória = {df.memory_usage(deep=True).sum()/1e6:.1f} MB")

# ==================== 5. SAMPLE (OPCIONAL) ===========================
if SAMPLE_FRAC:
    df = df.sample(frac=SAMPLE_FRAC, random_state=RANDOM_STATE)
    print(f"Amostra aleatória de {SAMPLE_FRAC:.0%}: {df.shape}")

# ==================== 6. FILTRO – participação e notas > 0 ===========
#  (agora converte presença/status p/ int  e compara com 1)
# =====================================================================

# Converte colunas de presença e status para inteiro (0/1/2)
int_cols = ["TP_PRESENCA_CN","TP_PRESENCA_CH","TP_PRESENCA_LC","TP_PRESENCA_MT",
            "TP_STATUS_REDACAO"]
df[int_cols] = df[int_cols].apply(pd.to_numeric, errors="coerce")

mask_pres = (df["TP_PRESENCA_CN"]==1)&(df["TP_PRESENCA_CH"]==1)&\
            (df["TP_PRESENCA_LC"]==1)&(df["TP_PRESENCA_MT"]==1)

mask_red  = df["TP_STATUS_REDACAO"].isin([1, 6])   # 1=corrigida, 6=vista pedagógica
mask_notas= (df[targets] > 0).all(axis=1)

df = df[mask_pres & mask_red & mask_notas]
print("Após filtro:", df.shape)       # → deve restar milhares de linhas

# ==================== 7. REFINO DAS FEATURES =========================
# Remove colunas de presença/status, agora irrelevantes
drop_cols = ["TP_PRESENCA_CN","TP_PRESENCA_CH","TP_PRESENCA_LC",
             "TP_PRESENCA_MT","TP_STATUS_REDACAO"]
cat_cols = [c for c in socio_cols if c not in drop_cols]

# Garante que todo categórico seja string
for col in cat_cols:
    df[col] = df[col].astype(str)

print(f"Total de variáveis socioeconômicas usadas: {len(cat_cols)}")

# ==================== 8. SPLIT TREINO / TESTE ========================
train, test = train_test_split(
    df,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=df["TP_FAIXA_ETARIA"]     # mantém distribuição etária
)

X_train, X_test = train[cat_cols], test[cat_cols]
Y_train, Y_test = train[targets],   test[targets]

print(f"Treino: {X_train.shape}  |  Teste: {X_test.shape}")

# ==================== 9. POOLS CATBOOST ==============================
pool_tr = Pool(X_train, Y_train, cat_features=cat_cols)
pool_te = Pool(X_test,  Y_test,  cat_features=cat_cols)

# ==================== 10. TREINO CATBOOST MULTITARGET ===============
model = CatBoostRegressor(
    loss_function="MultiRMSE",
    iterations=500,
    depth=7,
    learning_rate=0.07,
    random_state=RANDOM_STATE,
    verbose=100,
    early_stopping_rounds=50
)
model.fit(pool_tr, eval_set=pool_te)

# ==================== 11. METRICAS E IMPORTÂNCIA ====================
pred  = model.predict(pool_te)                    # matriz N × 5
mae   = np.mean(np.abs(pred - Y_test.values), axis=0)

print("\n── MAE por nota ──")
for n, v in zip(targets, mae):
    print(f"{n:13s}: {v:6.2f}")

print("\nTop‑15 variáveis socioeconômicas mais influentes (média das 5 notas):")
print(model.get_feature_importance(prettified=True).head(15).to_string(index=False))



✔️  Bibliotecas principais importadas
✔️  CSV carregado  |  shape = (3933955, 48)  |  memória = 330.6 MB
Amostra aleatória de 3%: (118019, 48)
Após filtro: (76972, 48)
Total de variáveis socioeconômicas usadas: 38
Treino: (61577, 38)  |  Teste: (15395, 38)
0:	learn: 250.6413866	test: 250.8372204	best: 250.8372204 (0)	total: 4.8s	remaining: 39m 53s
100:	learn: 215.3861660	test: 217.4346165	best: 217.4346165 (100)	total: 14m 2s	remaining: 55m 29s
200:	learn: 213.0916433	test: 216.6013898	best: 216.6013898 (200)	total: 30m 59s	remaining: 46m 5s
300:	learn: 211.7794585	test: 216.3812918	best: 216.3812918 (300)	total: 49m 46s	remaining: 32m 54s
400:	learn: 210.6535145	test: 216.2698623	best: 216.2696541 (395)	total: 1h 9m 47s	remaining: 17m 13s
499:	learn: 209.5893793	test: 216.2013858	best: 216.2013858 (499)	total: 1h 30m 50s	remaining: 0us

bestTest = 216.2013858
bestIteration = 499


── MAE por nota ──
NU_NOTA_CN   :  52.83
NU_NOTA_CH   :  57.13
NU_NOTA_LC   :  48.13
NU_NOTA_MT   :  80.8

In [1]:
#######################################################################
# ENEM 2023 – Predição simultânea das 5 notas finais (CN, CH, LC, MT,
# Redação) a partir de variáveis socioeconômicas, com CatBoost.
#######################################################################

# ==================== 1. IMPORTS =====================================
import warnings, sys
from pathlib import Path
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool

# ==================== 2. PARÂMETROS GLOBAIS ==========================
CSV_PATH     = Path(r"C:\wamp64\www\previsor-enem\dados\MICRODADOS_ENEM_2023.csv")
SAMPLE_FRAC  = 0.40        # None = 100 %, 0.80 = 80 %, 0.03 = 3 % protótipo
TEST_SIZE    = 0.20
RANDOM_STATE = 42

USE_GPU      = True       # True ⇒ usa GPU (CUDA); precisa driver e nvcc ≥ 10
MODEL_OUT    = Path("catboost_enem_multi.cbm")   # arquivo do modelo salvo

# ==================== 3. LISTAS DE VARIÁVEIS =========================
targets = ["NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC",
           "NU_NOTA_MT", "NU_NOTA_REDACAO"]

socio_cols = [
    # demografia / escola
    "TP_FAIXA_ETARIA","TP_SEXO","TP_ESTADO_CIVIL","TP_COR_RACA",
    "TP_NACIONALIDADE","TP_ST_CONCLUSAO","TP_ANO_CONCLUIU",
    "TP_ESCOLA","TP_ENSINO","IN_TREINEIRO","CO_MUNICIPIO_ESC",
    "TP_LOCALIZACAO_ESC","TP_SIT_FUNC_ESC",
    # questionário socioeconômico
    "Q001","Q002","Q003","Q004","Q005","Q006","Q007","Q008",
    "Q009","Q010","Q011","Q012","Q013","Q014","Q015","Q016",
    "Q017","Q018","Q019","Q020","Q021","Q022","Q023","Q024","Q025",
    # indicadores de participação (serão filtrados e depois removidos)
    "TP_PRESENCA_CN","TP_PRESENCA_CH","TP_PRESENCA_LC","TP_PRESENCA_MT",
    "TP_STATUS_REDACAO"
]

dtypes = {c: "category" for c in socio_cols}

# ==================== 4. LEITURA E SAMPLE ============================
df = pd.read_csv(
    CSV_PATH,
    sep=";",
    encoding="latin-1",
    usecols=socio_cols + targets,
    dtype=dtypes,
    engine="pyarrow"
)
print(f"CSV lido  →  linhas={df.shape[0]:,}  colunas={df.shape[1]}")

if SAMPLE_FRAC:
    df = df.sample(frac=SAMPLE_FRAC, random_state=RANDOM_STATE)
    print(f"Amostra de {SAMPLE_FRAC:.0%}: {df.shape[0]:,} linhas")

# ==================== 5. FILTRO DE PARTICIPAÇÃO / NOTA > 0 ===========
int_cols = ["TP_PRESENCA_CN","TP_PRESENCA_CH","TP_PRESENCA_LC",
            "TP_PRESENCA_MT","TP_STATUS_REDACAO"]
df[int_cols] = df[int_cols].apply(pd.to_numeric, errors="coerce")

mask_pres = (df["TP_PRESENCA_CN"]==1)&(df["TP_PRESENCA_CH"]==1)&\
            (df["TP_PRESENCA_LC"]==1)&(df["TP_PRESENCA_MT"]==1)
mask_red   = df["TP_STATUS_REDACAO"].isin([1, 6])
mask_notas = (df[targets] > 0).all(axis=1)

df = df[mask_pres & mask_red & mask_notas]
print(f"Após filtro participação/notas > 0: {df.shape[0]:,} linhas")

# ==================== 6. FEATURES FINAIS =============================
drop_cols = ["TP_PRESENCA_CN","TP_PRESENCA_CH","TP_PRESENCA_LC",
             "TP_PRESENCA_MT","TP_STATUS_REDACAO"]
cat_cols = [c for c in socio_cols if c not in drop_cols]

for c in cat_cols:
    df[c] = df[c].astype(str)

# ==================== 7. TRAIN / TEST SPLIT ==========================
train, test = train_test_split(
    df,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=df["TP_FAIXA_ETARIA"]
)
X_train, X_test = train[cat_cols], test[cat_cols]
Y_train, Y_test = train[targets],   test[targets]

print(f"Treino: {X_train.shape} | Teste: {X_test.shape}")

# ==================== 8. CATBOOST POOLS ==============================
pool_tr = Pool(X_train, Y_train, cat_features=cat_cols)
pool_te = Pool(X_test,  Y_test,  cat_features=cat_cols)

# ==================== 9. TREINO CATBOOST =============================
params = dict(
    loss_function="MultiRMSE",
    iterations=1000 if USE_GPU else 600,   # GPUs treinam rápido, podemos aumentar
    depth=8,
    learning_rate=0.05 if USE_GPU else 0.07,
    random_state=RANDOM_STATE,
    verbose=100,
    early_stopping_rounds=50,
    use_best_model=True,
)
if USE_GPU:
    params.update(dict(task_type="GPU", devices="0"))  # usa GPU 0

model = CatBoostRegressor(**params)
model.fit(pool_tr, eval_set=pool_te)

# ==================== 10. AVALIAÇÃO E SALVAMENTO =====================
pred = model.predict(pool_te)
mae  = np.abs(pred - Y_test.values).mean(axis=0)

print("\n── MAE por nota ──")
for n, v in zip(targets, mae):
    print(f"{n:15s}: {v:6.2f}")

print("\nSalvar modelo em:", MODEL_OUT)
model.save_model(MODEL_OUT)        # recarregue depois com CatBoostRegressor().load_model(...)

print("\nTop‑15 variáveis mais influentes (média das 5 notas):")
print(model.get_feature_importance(prettified=True).head(15).to_string(index=False))


CSV lido  →  linhas=3,933,955  colunas=48
Amostra de 40%: 1,573,582 linhas
Após filtro participação/notas > 0: 1,028,055 linhas
Treino: (822444, 38) | Teste: (205611, 38)
0:	learn: 251.4507539	test: 251.3304422	best: 251.3304422 (0)	total: 270ms	remaining: 4m 29s
100:	learn: 217.4743623	test: 217.4726104	best: 217.4726104 (100)	total: 45.6s	remaining: 6m 45s
200:	learn: 215.9974846	test: 216.1768061	best: 216.1768061 (200)	total: 2m 45s	remaining: 10m 56s
300:	learn: 215.2448690	test: 215.5954556	best: 215.5954556 (300)	total: 5m 11s	remaining: 12m 3s
400:	learn: 214.7352123	test: 215.2671147	best: 215.2671147 (400)	total: 7m 47s	remaining: 11m 38s
500:	learn: 214.3531911	test: 215.0739868	best: 215.0739868 (500)	total: 10m 18s	remaining: 10m 16s
600:	learn: 214.0559152	test: 214.9435436	best: 214.9435436 (600)	total: 13m 3s	remaining: 8m 40s
700:	learn: 213.8107114	test: 214.8475977	best: 214.8475977 (700)	total: 15m 39s	remaining: 6m 40s
800:	learn: 213.5889525	test: 214.7695579	best

In [3]:
# ==================== 11. AVALIAÇÃO DETALHADA ========================
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    median_absolute_error,
    r2_score,
    explained_variance_score,
)
import pandas as pd
import numpy as np

rows = []
for i, col in enumerate(targets):
    y_true = Y_test[col].values
    y_pred = pred[:, i]

    mae   = mean_absolute_error(y_true, y_pred)
    rmse  = mean_squared_error(y_true, y_pred, squared=False)
    r2    = r2_score(y_true, y_pred)
    medae = median_absolute_error(y_true, y_pred)
    evs   = explained_variance_score(y_true, y_pred)

    rows.append((col, mae, rmse, medae, r2, evs))

df_metrics = (
    pd.DataFrame(rows, columns=["Nota", "MAE", "RMSE", "MedAE", "R²", "ExpVar"])
      .round(2)
)
print("\n── Métricas por nota ──")
print(df_metrics.to_string(index=False))

# médias macro
macro = df_metrics.mean(numeric_only=True)
print("\nMédias macro (5 notas):")
print(f"MAE   : {round(macro['MAE'],   2)}")
print(f"RMSE  : {round(macro['RMSE'],  2)}")
print(f"MedAE : {round(macro['MedAE'], 2)}")
print(f"R²    : {round(macro['R²'],    3)}")
print(f"ExpVar: {round(macro['ExpVar'],3)}")



── Métricas por nota ──
           Nota    MAE   RMSE  MedAE   R²  ExpVar
     NU_NOTA_CN  52.35  65.72  44.43 0.30    0.30
     NU_NOTA_CH  56.68  71.95  46.89 0.26    0.26
     NU_NOTA_LC  48.16  61.56  39.62 0.27    0.27
     NU_NOTA_MT  80.83  99.57  70.90 0.36    0.36
NU_NOTA_REDACAO 122.04 151.27 104.60 0.25    0.25

Médias macro (5 notas):
MAE   : 72.01
RMSE  : 90.01
MedAE : 61.29
R²    : 0.288
ExpVar: 0.288
